In [1]:
# First import the necessary libraries
import os
import tensorflow as tf
import matplotlib.pyplot as plt

2024-11-05 21:41:18.140424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_dir = 'cats_and_dogs_filtered'

train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

print("Contents of base_dir:",base_dir)
print("Contents of train_dir:",train_dir)
print("Contents of validation_dir:",validation_dir)
print()
print("Contents of validation_cats_dir:",validation_cats_dir)
print("Contents of validation_dogs_dir:",validation_dogs_dir)
print()

Contents of base_dir: cats_and_dogs_filtered
Contents of train_dir: cats_and_dogs_filtered/train
Contents of validation_dir: cats_and_dogs_filtered/validation

Contents of validation_cats_dir: cats_and_dogs_filtered/validation/cats
Contents of validation_dogs_dir: cats_and_dogs_filtered/validation/dogs



In [5]:
train_cats_fnames = os.listdir(train_cats_dir)
train_dogs_fnames = os.listdir(train_dogs_dir)

validation_cats_fnames = os.listdir(validation_cats_dir)
validation_dogs_fnames = os.listdir(validation_dogs_dir)

In [6]:
print("train_cats_fnames[0:5]:",train_cats_fnames[0:5])
print("train_dogs_fnames[0:5]:",train_dogs_fnames[0:5])
print()
print("validation_cats_fnames[0:5]:",validation_cats_fnames[0:5])
print("validation_dogs_fnames[0:5]:",validation_dogs_fnames[0:5])

train_cats_fnames[0:5]: ['cat.803.jpg', 'cat.464.jpg', 'cat.882.jpg', 'cat.779.jpg', 'cat.194.jpg']
train_dogs_fnames[0:5]: ['dog.917.jpg', 'dog.795.jpg', 'dog.734.jpg', 'dog.715.jpg', 'dog.105.jpg']

validation_cats_fnames[0:5]: ['cat.2270.jpg', 'cat.2321.jpg', 'cat.2216.jpg', 'cat.2414.jpg', 'cat.2498.jpg']
validation_dogs_fnames[0:5]: ['dog.2141.jpg', 'dog.2449.jpg', 'dog.2363.jpg', 'dog.2097.jpg', 'dog.2038.jpg']


In [8]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape = (150,150,3)),
        tf.keras.layers.Rescaling(.1/255),
        
        tf.keras.layers.Conv2D(16,(3,3),activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
        tf.keras.layers.MaxPooling2D((2,2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512,activation = 'relu'),
        tf.keras.layers.Dense(1,activation = 'sigmoid')
    ])

    return model

In [9]:
untrained_model = create_model()

2024-11-05 21:58:59.814534: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
print(untrained_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        1

In [13]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size =  (150,150),
    batch_size = 20,
    label_mode = 'binary'
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size = (150,150),
    batch_size = 20,
    label_mode = 'binary'
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [14]:
shuffle_buffer_size = 1000
prefetch_buffer_size = tf.data.AUTOTUNE

train_dataset_final = train_dataset.cache().shuffle(buffer_size = shuffle_buffer_size).prefetch(buffer_size = prefetch_buffer_size)
validation_dataset_final = validation_dataset.cache().shuffle(buffer_size = shuffle_buffer_size).prefetch(buffer_size = prefetch_buffer_size)

In [15]:
model = untrained_model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)